In [4]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq


# setup the tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def square(a) -> int:
    """Calculates the square of a number."""
    a = int(a)
    return a * a

@tool
def give_increment(a) -> int:
    """if the number is negative then make it postive and return the number * 1.25"""
    a = abs(a)
    a = a * 1.25
    return a

@tool
def give_customise_increment(a, b) -> int:
    """if the number is negative then make it postive and return the number * b/100"""
    a = abs(a)
    a = a * b/100
    return a

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a mathematical assistant.
        Use your tools to answer questions. If you do not have a tool to
        answer the question, say so. 

        Return answer with description in a new line, e.g
        Human: What is 1 + 1?
        AI: 1 + 1 = 2
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Choose the LLM that will drive the agent
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

# setup the toolkit
toolkit = [add, multiply, square, give_increment, give_customise_increment]

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

result = agent_executor.invoke({"input": "what is 1 + 1?"})
print(result['output'])

result = agent_executor.invoke({"input": "how much to give on increment on -300"})
print(result['output'])

result = agent_executor.invoke({"input": "how much to give on increment on 150"})
print(result['output'])

result = agent_executor.invoke({"input": "how much to give on increment on 1000 with 10%"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 1, 'b': 1}`


22

> Finished chain.
2


> Entering new AgentExecutor chain...

Invoking: `give_increment` with `{'a': -300}`


375.0375.0

> Finished chain.
375.0


> Entering new AgentExecutor chain...

Invoking: `give_increment` with `{'a': 150}`


187.5
Invoking: `give_customise_increment` with `{'a': 187.5, 'b': 25}`


46.875187.5 * 1.25 = 46.875

> Finished chain.
187.5 * 1.25 = 46.875


> Entering new AgentExecutor chain...

Invoking: `give_customise_increment` with `{'a': 1000, 'b': 10}`


100.0
Invoking: `give_customise_increment` with `{'a': 1000, 'b': 10}`


100.0The increment on 1000 with 10% is 100.0.

> Finished chain.
The increment on 1000 with 10% is 100.0.
